In [1]:
from simulation_utils_extended import * 
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import gpboost as gpb
#import pdpbox
#from pdpbox import pdp
sns.set()

In [ ]:
def train_and_test_Vecchia(datasets, n_datasets, num_boost_round=0, params={}, merf=False, GPBoost_cat=False, linear=False, GP=False, shared=False, no_features=False, kernel='exponential', 
                                                            matern_param=2.5, vecchia_approx=True, num_neighbors=5, vecchia_ordering='none', vecchia_pred_type='order_pred_first', num_neighbors_pred=5):

    np.random.seed(1)
    RMSE_list1 = []
    RMSE_list2 = []
    F_list = []
    time_list = []

    for i in tnrange(n_datasets):

        data_train = datasets['data_train'][i]
        X_train = datasets['X_train'][i]
        X_test1 = datasets['X_test1'][i]
        X_test2 = datasets['X_test2'][i]
        y_train = datasets['y_train'][i]
        y_test1 = datasets['y_test1'][i]
        y_test2 = datasets['y_test2'][i]
        F_test1 = datasets['F_test1'][i]
        F_test2 = datasets['F_test2'][i]


        #if not GPBoost_cat:
        # Random effects covariates
        groups_train = X_train['group']
        groups_test1 = X_test1['group']
        groups_test2 = X_test2['group']
        times_train = X_train['times']
        times_test1 = X_test1['times']
        times_test2 = X_test2['times']


        ########### Linear mixed effects model ##############
        if linear:
            if no_features:
                X_train_linear = np.ones(len(X_train))
                X_test1_linear = np.ones(len(X_test1))
                X_test2_linear = np.ones(len(X_test2))
            else:
                X_train_linear = np.column_stack((np.ones(len(X_train)), X_train[['feature_1','feature_2','feature_3','feature_4']])) # add column of ones to get a global intercept 
                X_test1_linear = np.column_stack((np.ones(len(X_test1)), X_test1[['feature_1','feature_2','feature_3','feature_4']]))
                X_test2_linear = np.column_stack((np.ones(len(X_test2)), X_test2[['feature_1','feature_2','feature_3','feature_4']]))
        #########################################################


        # Define the model
        if merf:
            Z_train = np.column_stack((np.ones(len(X_train)), times_train))
            Z_test1 = np.column_stack((np.ones(len(X_test1)), times_test1))
            Z_test2 = np.column_stack((np.ones(len(X_test2)), times_test2))
            fixed_effects_model = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=1)
            merf_model = MERF(fixed_effects_model, max_iterations=100)

        elif GPBoost_cat:
            pass 


        else:
            # Models with GP components for the observation times (includes both linear models and Boosted-tree fixed effect models)
            if GP:

                if kernel == 'matern':
                    if shared:
                        gp_model = gpb.GPModel(group_data=groups_train, gp_coords=times_train, cov_function=kernel, cov_fct_shape=matern_param)
                    else:
                        gp_model = gpb.GPModel(group_data=groups_train, gp_coords=times_train, cluster_ids=groups_train, cov_function=kernel, cov_fct_shape=matern_param)

                else:
                    if shared:
                        gp_model = gpb.GPModel(gp_coords=X_train['times'], cov_function='exponential', vecchia_approx=vecchia_approx, num_neighbors=num_neighbors, 
                                                                                vecchia_ordering=vecchia_ordering, vecchia_pred_type=vecchia_pred_type, num_neighbors_pred=num_neighbors_pred)
                    else:
                        gp_model = gpb.GPModel(group_data=groups_train, gp_coords=times_train, cluster_ids=groups_train, cov_function=kernel)

            # LME models without GP components
            else:
                gp_model = gpb.GPModel(group_data=np.column_stack((groups_train, times_train)))
            gp_model.set_optim_params(params={'optimizer_cov': 'gradient_descent', 'use_nesterov_acc': True}) # these are the default parameters


        #################### Training ####################
        start_time = time.time()

        try:
            if merf:
                merf_model.fit(X_train[['feature_1','feature_2','feature_3','feature_4']], Z_train, groups_train, y_train)

            elif GPBoost_cat:
                bst = gpb.train(params=params, train_set=data_train, num_boost_round=num_boost_round, categorical_feature=['group'])
                #bst = gpb.train(params=params, train_set=data_train, num_boost_round=num_boost_round, categorical_feature=['group'], feature_name=[col_name for col_name in datasets['X_train'][0].columns.values])

            else:
                if linear: # LME models
                    gp_model.fit(y=y_train, X=X_train_linear)
                else: # GPBoost models
                    bst = gpb.train(params=params, train_set=data_train, gp_model=gp_model, num_boost_round=num_boost_round)

            time_list.append(time.time() - start_time)


            #################### Prediction ####################
            if merf:
                y_pred1 = merf_model.predict(X_test1[['feature_1','feature_2','feature_3','feature_4']], Z_test1, groups_test1)
                y_pred2 = merf_model.predict(X_test2[['feature_1','feature_2','feature_3','feature_4']], Z_test2, groups_test2)
                F_pred1 = merf_model.trained_fe_model.predict(X_test1[['feature_1','feature_2','feature_3','feature_4']])
                F_pred2 = merf_model.trained_fe_model.predict(X_test2[['feature_1','feature_2','feature_3','feature_4']])

            elif GPBoost_cat:
                y_pred1 = bst.predict(data=X_test1)
                y_pred2 = bst.predict(data=X_test2)

            else:

                if linear: # LME models
                    if GP:
                        if shared:
                            y_pred1 = gp_model.predict(group_data_pred=groups_test1, gp_coords_pred=times_test1, X_pred=X_test1_linear)['mu']
                            y_pred2 = gp_model.predict(group_data_pred=groups_test2, gp_coords_pred=times_test2, X_pred=X_test2_linear)['mu']
                        else:
                            y_pred1 = gp_model.predict(group_data_pred=groups_test1, gp_coords_pred=times_test1, cluster_ids_pred=groups_test1, X_pred=X_test1_linear)['mu']
                            y_pred2 = gp_model.predict(group_data_pred=groups_test2, gp_coords_pred=times_test2, cluster_ids_pred=groups_test2, X_pred=X_test2_linear)['mu']
                    else:
                        y_pred1 = gp_model.predict(group_data_pred=np.column_stack((groups_test1, times_test1)), X_pred=X_test1_linear)['mu']
                        y_pred2 = gp_model.predict(group_data_pred=np.column_stack((groups_test2, times_test2)), X_pred=X_test2_linear)['mu']

                        

                    if not no_features: # Linear models Fixed effects part prediction, i.e. F(X) = Xβ for LME models
                        F_pred1 = X_test1_linear.dot(gp_model.get_coef().T)
                        F_pred2 = X_test2_linear.dot(gp_model.get_coef().T)

                else: # GPBoost models
                    if GP:
                        if shared:
                            pred1 = bst.predict(data=X_test1[['feature_1','feature_2','feature_3','feature_4']], gp_coords_pred=X_test1['times'], pred_latent=True)
                            pred2 = bst.predict(data=X_test2[['feature_1','feature_2','feature_3','feature_4']], gp_coords_pred=X_test2['times'], pred_latent=True)
                        else:
                            pred1 = bst.predict(data=X_test1[['feature_1','feature_2','feature_3','feature_4']], group_data_pred=groups_test1, gp_coords_pred=times_test1, cluster_ids_pred=groups_test1, pred_latent=True)
                            pred2 = bst.predict(data=X_test2[['feature_1','feature_2','feature_3','feature_4']], group_data_pred=groups_test2, gp_coords_pred=times_test2, cluster_ids_pred=groups_test2, pred_latent=True)
                    else:
                        pred1 = bst.predict(data=X_test1[['feature_1','feature_2','feature_3','feature_4']], group_data_pred=np.column_stack((groups_test1, times_test1)), pred_latent=True)
                        pred2 = bst.predict(data=X_test2[['feature_1','feature_2','feature_3','feature_4']], group_data_pred=np.column_stack((groups_test2, times_test2)), pred_latent=True)

                        

                    y_pred1 = pred1['fixed_effect'] + pred1['random_effect_mean']
                    y_pred2 = pred2['fixed_effect'] + pred2['random_effect_mean']


                    if not no_features: # Fixed effects part prediction
                        F_pred1, F_pred2 = pred1['fixed_effect'], pred2['fixed_effect']



            RMSE_list1.append(np.sqrt(np.mean((y_test1-y_pred1)**2)))
            RMSE_list2.append(np.sqrt(np.mean((y_test2-y_pred2)**2)))

            if not no_features:
                F_list.append(np.sqrt(np.mean((F_test1-F_pred1)**2)))
                F_list.append(np.sqrt(np.mean((F_test2-F_pred2)**2))) # RMSE for F is given by both interpolation and extrapolation

        except:
            print('Error encountered')
            continue  
    
    #print(gp_model.get_cov_pars().values.squeeze())
    if no_features: # fixed part prediction is just the global average target value
        return time_list, RMSE_list1, RMSE_list2
    else:
        return time_list, RMSE_list1, RMSE_list2, F_list

In [2]:
# Generate data
n, m = 500, 50  # Number of observations and groups
p = int(n/m) # Number of observations per group
n_datasets = 20 
n_valid = 5
datasets, validation_datasets = generate_datasets(n, m, p, n_datasets, n_valid, func='make_friedman3', random_state=60, shared_gp=True, rho=0.1, sigma2_2=1)

# Create groups
groups = np.arange(n)
for i in range(m):
    groups[i*p:(i+1)*p] = i

# Create times for validation datasets
times = np.tile(np.arange(p)*10+10, m)

# Create times for validation datasets
times = np.tile(np.arange(p)*10+10, m)

X_train = datasets['X_train'][0]
F_train = datasets['F_train'][0]
y_train = datasets['y_train'][0]
X_test_int = datasets['X_test2'][0]
X_test_ext = datasets['X_test1'][0]
y_test_int = datasets['y_test2'][0]
y_test_ext = datasets['y_test1'][0]
F_test_int = datasets['F_test2'][0]
F_test_ext = datasets['F_test1'][0]
full_data = datasets['data'][0]
full_data.head()

,group,times,feature_1,feature_2,feature_3,feature_4,y
0,0,18,30.087333,431.063660,0.323183,7.657496,2.134189
1,0,38,56.697080,776.262596,0.379415,1.105815,3.009979
2,0,120,17.036560,327.242594,0.692401,9.744416,5.598567
3,0,224,33.739690,1746.973065,0.131540,6.003298,3.262823
4,0,272,28.662051,486.017341,0.502086,7.360625,1.711149


In [3]:
# Create results dataframe
results_1 = pd.DataFrame(columns=["RMSE Interpolation (mean)", "RMSE Interpolation (std)", "RMSE Extrapolation (mean)", "RMSE Extrapolation (std)", "RMSE_F (mean)", "RMSE_F (std)", "Time"],
                         index=['Linear Mixed Effects Model with no fixed features (random intercept)',
                                   'Linear Mixed Effects Model with Random Intercept',
                                   'Linear Mixed Effects Model with Shared Gaussian Process',
                                   'Linear Mixed Effects Model with Independent Gaussian Process',
                                   'Gradient-boosted tree with group as categorical variable (no random effects)',
                                   'GPBoost with Random Intercept',
                                   'GPBoost with Shared Gaussian Process',
                                   'GPBoost with Independent Gaussian Process'])

In [14]:
# 7. GPBoost with Shared Gaussian Process using Vecchia approximation
params = {'learning_rate': 0.1,
          'max_depth': 3,
          'min_data_in_leaf': 10,
          'objective': 'regression_l2',                                                                                                                         
          'verbose': 0,                                                                                                                                 
          'num_leaves': 2**10}

time_list, RMSE_list1, RMSE_list2, F_list = train_and_test_Vecchia(datasets, n_datasets, num_boost_round=57, params=params, num_neighbors=10, vecchia_ordering='none', vecchia_pred_type='order_pred_first', num_neighbors_pred=10,
                                                            linear=False, GP=True, shared=True, kernel='exponential');
                                                            
results_1.loc["GPBoost with Shared Gaussian Process", "Time"] = np.mean(time_list)
results_1.loc["GPBoost with Shared Gaussian Process", "RMSE Extrapolation (mean)"] = np.mean(RMSE_list1)
results_1.loc["GPBoost with Shared Gaussian Process", "RMSE Extrapolation (std)"] = np.std(RMSE_list1)
results_1.loc["GPBoost with Shared Gaussian Process", "RMSE Interpolation (mean)"] = np.mean(RMSE_list2)
results_1.loc["GPBoost with Shared Gaussian Process", "RMSE Interpolation (std)"] = np.std(RMSE_list2)
results_1.loc["GPBoost with Shared Gaussian Process", "RMSE_F (mean)"] = np.mean(F_list)
results_1.loc["GPBoost with Shared Gaussian Process", "RMSE_F (std)"] = np.std(F_list)
results_1

  0%|          | 0/20 [00:00<?, ?it/s]

[GPBoost] [Info] Starting nearest neighbor search for Vecchia approximation
[GPBoost] [Info] Nearest neighbors for Vecchia approximation found
[GPBoost] [Info] Starting nearest neighbor search for Vecchia approximation
[GPBoost] [Info] Nearest neighbors for Vecchia approximation found
[GPBoost] [Info] Starting nearest neighbor search for Vecchia approximation
[GPBoost] [Info] Nearest neighbors for Vecchia approximation found
[GPBoost] [Info] Starting nearest neighbor search for Vecchia approximation
[GPBoost] [Info] Nearest neighbors for Vecchia approximation found
[GPBoost] [Info] Starting nearest neighbor search for Vecchia approximation
[GPBoost] [Info] Nearest neighbors for Vecchia approximation found
[GPBoost] [Info] Starting nearest neighbor search for Vecchia approximation
[GPBoost] [Info] Nearest neighbors for Vecchia approximation found
[GPBoost] [Info] Starting nearest neighbor search for Vecchia approximation
[GPBoost] [Info] Nearest neighbors for Vecchia approximation found

,RMSE Interpolation (mean),RMSE Interpolation (std),RMSE Extrapolation (mean),RMSE Extrapolation (std),RMSE_F (mean),RMSE_F (std),Time
Linear Mixed Effects Model with no fixed features (random intercept),NaN,NaN,NaN,NaN,NaN,NaN,NaN
Linear Mixed Effects Model with Random Intercept,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Linear Mixed Effects Model with Shared Gaussian Process,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Linear Mixed Effects Model with Independent Gaussian Process,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gradient-boosted tree with group as categorical variable (no random effects),NaN,NaN,NaN,NaN,NaN,NaN,NaN
GPBoost with Random Intercept,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GPBoost with Shared Gaussian Process,1.658098,0.135292,1.669791,0.166917,1.563295,0.171859,0.324683
GPBoost with Independent Gaussian Process,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## NOTE: Vecchia approximation can't be used with grouped random effects (ie. can't have a random intercept in the random effects model)

In [16]:
params = {'learning_rate': 0.1,
          'max_depth': 3,
          'min_data_in_leaf': 10,
          'objective': 'regression_l2',
          'verbose': 0,
          'num_leaves': 2**10}

data_train = gpb.Dataset(data=X_train[['feature_1','feature_2','feature_3','feature_4']], label=y_train)
gp_model = gpb.GPModel(gp_coords=X_train['times'], cov_function='exponential', vecchia_approx=True, num_neighbors=5, vecchia_ordering='none', vecchia_pred_type='order_pred_first', num_neighbors_pred=5)
gp_model.set_optim_params(params={'optimizer_cov': 'gradient_descent', 'use_nesterov_acc': True})


bst = gpb.train(params=params, train_set=data_train, gp_model=gp_model, num_boost_round=57)
pred1 = bst.predict(data=X_test_ext[['feature_1','feature_2','feature_3','feature_4']], gp_coords_pred=X_test_ext['times'], pred_latent=True)
pred2 = bst.predict(data=X_test_int[['feature_1','feature_2','feature_3','feature_4']], gp_coords_pred=X_test_int['times'], pred_latent=True)

[GPBoost] [Info] Starting nearest neighbor search for Vecchia approximation
[GPBoost] [Info] Nearest neighbors for Vecchia approximation found
